## Purpose

Recovery / Resume demonstration

This notebook shows how to **resume** a pipeline from its saved trace, and how the
library **safely refuses** to resume if the new pipeline points to a *different*
outputs directory.

We include two short demos:
1. **Incorrect resume** → raises `ResumeExecutionError` (mismatched `outputs_directory`).
2. **Correct resume** → continues from the previous trace and appends a new step.

## Dependencies Installation


In [ ]:
import isx
from ci_pipe.pipeline import CIPipe

In [ ]:
import os
upper_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
requirements_path = os.path.join(upper_root, "requirements.txt")
!pip install -r $requirements_path

## Pipeline creation

In [ ]:
pipeline = CIPipe.with_videos_from_directory('input_dir', outputs_directory='output_dir', isx=isx)

### Half pipeline execution with ISX algorithms


In [ ]:
(
    pipeline
    .set_defaults(isx_bp_subtract_global_minimum=False, isx_mc_max_translation=25, isx_acr_filters=[('SNR', '>', 3), ('Event Rate', '>', 0), ('# Comps', '=', 1)])
    .isx.preprocess_videos()
    .isx.bandpass_filter_videos()
    .isx.motion_correction_videos(isx_mc_series_name="series1")
)

### New pipeline creation with different output directory


In [ ]:
another_pipeline = CIPipe.with_videos_from_directory('input_dir', outputs_directory='new_output_dir', isx=isx)


### Attempt to resume from trace with different outputs directory

In [ ]:
from ci_pipe.errors.resume_execution_error import ResumeExecutionError

try:
    (
        another_pipeline
        .isx.normalize_dff_videos()
        .isx.extract_neurons_pca_ica()
        .isx.detect_events_in_cells()
        .isx.auto_accept_reject_cells()
    )
except ResumeExecutionError as error:
    print(error)


Note: Since we are internally setting the trace file when creating the pipeline there's currently no way the user could inject a different trace name and get to failure when resuming.

### New pipeline creation with the same output directory

In [ ]:
valid_pipeline = CIPipe.with_videos_from_directory('input_dir', outputs_directory='output_dir', isx=isx)


In [ ]:
(
    valid_pipeline
    .isx.normalize_dff_videos()
    .isx.extract_neurons_pca_ica()
    .isx.detect_events_in_cells()
    .isx.auto_accept_reject_cells()
)